In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

from scipy.stats import f
from scipy import linalg

In [2]:
data_dir = "./data"

In [3]:
ff_5_factors = pd.read_csv(f"{data_dir}/F-F_Research_Data_5_Factors_2x3.csv", skiprows=4, names=["Year_Month", "Mkt", "SMB", "HML", "RMW", "CMA", "RF"])
ff_5_factors = ff_5_factors.iloc[:636]
'''The first concerns the profitability factor RMWO in Eq. (4). It is constructed by sorting stocks
on the accruals-based operating profitability (OP) measure
suggested by Novy-Marx (2013), and it is the profitability factor in the five-factor model of FF (2015, 2016, 2017)
'''
ff_mom = pd.read_csv(f"{data_dir}/F-F_Momentum_Factor.csv", skiprows=14, names=["Year_Month", "Mom"])

formed_on_size = pd.read_csv(f"{data_dir}/Portfolios_Formed_on_ME.csv", skiprows=13, usecols=range(5), names=["Year_Month", "<= 0", "Lo 30", "Med 40", "Hi 30"])

In [4]:
ff_mom

,Year_Month,Mom
0,192701,0.36
1,192702,-2.14
2,192703,3.61
3,192704,4.30
4,192705,3.00
...,...,...
1265,2020,7.75
1266,2021,-2.40
1267,2022,15.50
1268,2023,-24.17


In [5]:
ff_6_factors = ff_5_factors.merge(ff_mom, on='Year_Month', how='left')
ff_6_factors

,Year_Month,Mkt,SMB,HML,RMW,CMA,RF,Mom
0,196307,-0.39,-0.41,-0.97,0.68,-1.18,0.27,0.90
1,196308,5.07,-0.80,1.80,0.36,-0.35,0.25,1.01
2,196309,-1.57,-0.52,0.13,-0.71,0.29,0.27,0.19
3,196310,2.53,-1.39,-0.10,2.80,-2.01,0.29,3.12
4,196311,-0.85,-0.88,1.75,-0.51,2.24,0.27,-0.74
...,...,...,...,...,...,...,...,...
631,201602,-0.07,0.88,-0.57,3.25,2.02,0.02,-4.38
632,201603,6.96,1.07,1.19,0.77,-0.08,0.02,-5.01
633,201604,0.91,1.23,3.28,-2.97,1.90,0.01,-6.02
634,201605,1.78,-0.61,-1.66,-1.09,-2.48,0.01,1.42


In [6]:
ff_6_factors[["Mkt", "SMB", "HML", "RMW", "CMA", "RF", "Mom"]] = ff_6_factors[["Mkt", "SMB", "HML", "RMW", "CMA", "RF", "Mom"]].astype(float)

# ff_5_factors["Year_Month"] = pd.to_datetime(ff_5_factors["Year_Month"], format='%Y%m').dt.strftime('%Y%m')
ff_6_factors 

,Year_Month,Mkt,SMB,HML,RMW,CMA,RF,Mom
0,196307,-0.39,-0.41,-0.97,0.68,-1.18,0.27,0.90
1,196308,5.07,-0.80,1.80,0.36,-0.35,0.25,1.01
2,196309,-1.57,-0.52,0.13,-0.71,0.29,0.27,0.19
3,196310,2.53,-1.39,-0.10,2.80,-2.01,0.29,3.12
4,196311,-0.85,-0.88,1.75,-0.51,2.24,0.27,-0.74
...,...,...,...,...,...,...,...,...
631,201602,-0.07,0.88,-0.57,3.25,2.02,0.02,-4.38
632,201603,6.96,1.07,1.19,0.77,-0.08,0.02,-5.01
633,201604,0.91,1.23,3.28,-2.97,1.90,0.01,-6.02
634,201605,1.78,-0.61,-1.66,-1.09,-2.48,0.01,1.42


In [7]:
ff_6_factors.describe()

,Mkt,SMB,HML,RMW,CMA,RF,Mom
count,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000,636.000000
mean,0.500487,0.250220,0.344937,0.264135,0.310079,0.395865,0.690991
std,4.437925,3.035771,2.794975,2.222850,1.989572,0.262893,4.235675
min,-23.240000,-15.320000,-11.290000,-18.650000,-6.800000,0.000000,-34.300000
25%,-1.982500,-1.465000,-1.222500,-0.780000,-0.950000,0.250000,-0.740000
50%,0.805000,0.100000,0.315000,0.235000,0.140000,0.400000,0.780000
75%,3.407500,2.072500,1.730000,1.302500,1.520000,0.530000,2.922500
max,16.100000,18.280000,12.470000,13.070000,9.070000,1.350000,18.200000


In [8]:
ff_6_factors = ff_6_factors.drop(['RF'], axis=1)

In [9]:
ff_6_factors = ff_6_factors.set_index('Year_Month')

### Table 2

In [10]:

# Function to perform spanning regression
def spanning_regression(formula, data):

    model = smf.ols(formula, data=data).fit()
    return model

def get_regression_params(model):
    coef = model.params
    t_val = model.tvalues
    r_squared = model.rsquared
    std_error = model.resid.std()

    return coef, t_val, r_squared, std_error

# Perform regressions
capm_smb = spanning_regression('SMB ~ Mkt', ff_6_factors)
capm_hml = spanning_regression('HML ~ Mkt', ff_6_factors)
three_factor_rmwo = spanning_regression('RMW ~ Mkt + SMB + HML', ff_6_factors)
three_factor_cma = spanning_regression('CMA ~ Mkt + SMB + HML', ff_6_factors)
five_factor_umd = spanning_regression('Mom ~ Mkt + SMB + HML + RMW + CMA', ff_6_factors)


In [11]:
# Get parameters for each model
models = [capm_smb, capm_hml, three_factor_rmwo, three_factor_cma, five_factor_umd]
model_names = ['SMB', 'HML', 'RMW', 'CMA', 'UMD']

results = []

for model, name in zip(models, model_names):
    coef, t_val, r_squared, std_error = get_regression_params(model)
    row = {
        'LHS': name,
        'Int': coef.get('Intercept', None),
        'Mkt': coef.get('Mkt', None),
        'SMB': coef.get('SMB', None),
        'HML': coef.get('HML', None),
        'RMW': coef.get('RMW', None),
        'CMA': coef.get('CMA', None),
        'UMD': coef.get('UMD', None),
        'Int_t': t_val.get('Intercept', None),
        'Mkt_t': t_val.get('Mkt', None),
        'SMB_t': t_val.get('SMB', None),
        'HML_t': t_val.get('HML', None),
        'RMW_t': t_val.get('RMW', None),
        'CMA_t': t_val.get('CMA', None),
        'UMD_t': t_val.get('UMD', None),
        'R2': r_squared,
        's(e)': std_error
    }
    results.append(row)

panel_a = pd.DataFrame(results)
panel_a = panel_a[['LHS', 'Int', 'Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'Int_t', 'Mkt_t', 'SMB_t', 'HML_t', 'RMW_t', 'CMA_t', 'R2', 's(e)']]

panel_a

,LHS,Int,Mkt,SMB,HML,RMW,CMA,Int_t,Mkt_t,SMB_t,HML_t,RMW_t,CMA_t,R2,s(e)
0,SMB,0.156380,0.187498,NaN,NaN,NaN,NaN,1.341248,7.176486,NaN,NaN,NaN,NaN,0.075130,2.921807
1,HML,0.427689,-0.165342,NaN,NaN,NaN,NaN,3.970958,-6.850757,NaN,NaN,NaN,NaN,0.068924,2.699061
2,RMW,0.353430,-0.070840,-0.226442,0.008175,NaN,NaN,4.230362,-3.571691,-8.066693,0.269017,NaN,NaN,0.141007,2.065064
3,CMA,0.205160,-0.100894,-0.001349,0.451538,NaN,NaN,3.702948,-7.670825,-0.072441,22.406575,NaN,NaN,0.528448,1.369470
4,UMD,0.745395,-0.132139,0.068484,-0.515934,0.236206,0.35529,4.398980,-3.174926,1.178385,-6.367745,2.947042,2.939666,0.088168,4.060660


In [12]:
panel_a.to_excel('./result/table2_a.xlsx', index=False)

In [13]:
def grs_test(resid: np.ndarray, alpha: np.ndarray, factors: np.ndarray) -> tuple:
    """ Perform the Gibbons, Ross and Shaken (1989) test.
        :param resid: Matrix of residuals from the OLS of size TxK.
        :param alpha: Vector of alphas from the OLS of size Kx1.
        :param factors: Matrix of factor returns of size TxJ.
        :return Test statistic and pValue of the test statistic.

        source: https://github.com/SteffenGue/GRS_Test/blob/main/GRSTest.py
    """
    # Determine the time series and assets
    iT, iK = resid.shape

    # Determine the amount of risk factors
    iJ = factors.shape[1]

    # Input size checks
    assert alpha.shape == (iK, 1)
    assert factors.shape == (iT, iJ)

    # Covariance of the residuals, variables are in columns.
    mCov = np.cov(resid, rowvar=False)

    # Mean of excess returns of the risk factors
    vMuRF = np.nanmean(factors, axis=0)

    try:
        assert vMuRF.shape == (1, iJ)
    except AssertionError:
        vMuRF = vMuRF.reshape(1, iJ)

    # Duplicate this series for T timestamps
    mMuRF = np.repeat(vMuRF, iT, axis=0)

    # Test statistic
    mCovRF = (factors - mMuRF).T @ (factors - mMuRF) / (iT - 1)

    # mCov = mCov.reshape(-1, 1)
    dTestStat = (iT / iK) * ((iT - iK - iJ) / (iT - iJ - 1)) * \
                (alpha.T @ (np.linalg.inv(mCov) @ alpha)) / \
                (1 + (vMuRF @ (np.linalg.inv(mCovRF) @ vMuRF.T)))

    pVal = 1 - f.cdf(dTestStat, iK, iT-iK-1)

    return dTestStat, pVal

def model_grs(models):
    alphas = []
    residuals = []
    
    for model in models:
        alphas.append(model.params['Intercept'])
        residuals.append(model.resid)

    alphas = np.array(alphas).reshape(-1, 1)
    residuals = np.array(residuals).T
    
    grs, p_value = grs_test(residuals, alphas, ff_6_factors.to_numpy())

    return grs, p_value

In [14]:
capm_models = [capm_smb, capm_hml]
three_factor_models = [three_factor_rmwo, three_factor_cma]

models_b = [capm_models, three_factor_models]
model_b_names = [['CAPM', 'SMB, HML'], ['Three-factor model', 'RMWO, CMA']]

results_b = []

for model, name in zip(models_b, model_b_names):
    grs, p_value = model_grs(model)
    row = {
        'Model': name[0],
        'LHS returns': name[1],
        'GRS': round(grs[0][0], 2),
        'p-value': round(p_value[0][0], 3),
    }
    results_b.append(row)

panel_b = pd.DataFrame(results_b)
panel_b

,Model,LHS returns,GRS,p-value
0,CAPM,"SMB, HML",7.84,0.0
1,Three-factor model,"RMWO, CMA",18.47,0.0


In [15]:
panel_b.to_excel('./result/table2_b.xlsx', index=False)